In [1]:
from IPython.core.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 jQuery('div.input').hide();
 } else {
 jQuery('div.input').show();
 }
 code_show = !code_show
} 
jQuery( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle code"></form>''')

# Huckel Code application

In order to run the application, execute all cells from top to bottom by pressing __shift + enter__.

You may interrupt execution by clicking on the __interrupt kernel__ button.

## Loading modules

In [1]:
import os
import subprocess
import time
import fnmatch
from subprocess import Popen, PIPE
from IPython.display import display, clear_output, Image
from ipywidgets import interact, interactive, fixed, interact_manual, Checkbox
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, SelectMultiple, Checkbox, interactive, FloatRangeSlider, Accordion, HTML
from IPython.display import display

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import gc

from IPython.core.display import Image

In [2]:
#Group of checkboces for configuration properties
distance_mx = Checkbox(
    description=' ',
    value=False
)
overlap_pop = Checkbox(
    value=False,
    description=' '
)
red_overlap_pop = Checkbox(
    value=False,
    description=' '
)
charge_mx = Checkbox(
    value=False,
    description=' '
)
wave_func = Checkbox(
    value=False,
    description=' '
)
net_charges = Checkbox(
    value=False,
    description=' '
)
overlap = Checkbox(
    value=False,
    description=' '
)

hamil = Checkbox(
    value=False,
    description=' '
)

electrostatic = Checkbox(
    value=False,
    description=' '
)

levels = Checkbox(
    value=False,
    description=' '
)

fermi = Checkbox(
    value=False,
    description=' '
)

orbital_energy = Checkbox(
    value=False,
    description=' '
)

orbital_coeff = Checkbox(
    value=False,
    description=' '
)

orbital_mapping = Checkbox(
    value=False,
    description=' '
)

dump_overlap = Checkbox(
    value=False,
    description=' '
)

dump_hamil = Checkbox(
    value=False,
    description=' '
)

dump_sparse = Checkbox(
    value=False,
    description=' '
)

dump_mm = Checkbox(
    value=False,
    description=' '
)

#List of property checkboxes
all_checkboxes = [distance_mx, overlap_pop,red_overlap_pop, charge_mx,
              wave_func,net_charges,overlap, hamil, electrostatic, 
                  levels, fermi,orbital_energy, orbital_coeff,
              orbital_mapping,dump_overlap,dump_hamil, dump_sparse, dump_mm]

#List of property labels
all_labels = ['Distance matrix', 'Overlap population', 'Reduced overlap population', 'Charge matrix',
             'Wave functions', 'Net charges', 'Overlap matrix', 'Hamiltonian matrix', 'Electrostatic',
             'Levels', 'Fermi', 'Orbital energy', 'Orbital coefficient', 'Orbital mapping', 'Dump overlap',
             'Dump Hamiltonian', 'Dump sparse', 'Dump Matrix Market']

#List of property keywords
all_properties = ['distance matrix', 'overlap population','reduced overlap population','charge matrix',
                  'wave functions','net charges','overlap','hamil','electrostatic','levels','fermi',
                  'orbital energy','orbital coeff','orbital mapping','dump overlap',
                  'dump hamil','dump sparse','dump matrix market']

firstgroup = [distance_mx, overlap_pop,red_overlap_pop, charge_mx,
              wave_func,net_charges,overlap, hamil, electrostatic]


secondgroup = [levels, fermi,orbital_energy, orbital_coeff,
              orbital_mapping,dump_overlap,dump_hamil, dump_sparse]


In [3]:
#Function for creating input
def createinput( arg1, arg2 ):
    process = Popen(["./input_pdb", arg1, arg2], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (output, err) = process.communicate()
    exit_code = process.wait()
    if err ==  b'':
        print('Creation of input file was successful')
        return
    else:
        return err.strip()

#Function for calculations
def huckel(arg):
    process = Popen(["./bind", arg], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (output, err) = process.communicate()
    exit_code = process.wait()
    if err ==  b'':
        print('Extended Huckel calculations were successful')
        return
    else:
        return err

## Please specify input file names
_If non-existent, configfile will be written function based on user input of the checkboxes by running cell containing ``writeconfigfile()``_

In [4]:
molFileName = input("Molecule filename: ")
configFileName = input("Configuration filename: ")
molName = molFileName[:molFileName.find('.')]
yahFileName= molName + '.yah'

Molecule filename: 1bna.pdb
Configuration filename: config.txt


In [5]:
#Layout for showing the checkboxes of configuration options
label_layout = Layout()

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
#     width='50%'
)

form_mainlabel_layout = Layout(width='180px')

form_items = [Box([HTML('<b><font size="4">Please set configuration parameters by clicking the checkboxes below:</font></b>')])]

if (len(all_checkboxes)%2 == 0):
    for cb_index in range(int(np.ceil(len(all_checkboxes)/2))):
        form_items.append(Box([Label(value=all_labels[cb_index], layout=form_mainlabel_layout), all_checkboxes[cb_index], Label(value=all_labels[cb_index+int(np.ceil(len(all_checkboxes)/2))], layout=form_mainlabel_layout), all_checkboxes[cb_index+int(np.ceil(len(all_checkboxes)/2))]], layout=form_item_layout))
else:
    for cb_index in range(int(np.floor(len(all_checkboxes)/2))):
        form_items.append(Box([Label(value=all_labels[cb_index], layout=form_mainlabel_layout), all_checkboxes[cb_index], Label(value=all_labels[cb_index+int(np.ceil(len(all_checkboxes)/2))], layout=form_mainlabel_layout), all_checkboxes[cb_index+int(np.ceil(len(all_checkboxes)/2))]], layout=form_item_layout))
    form_items.append(Box([Label(value=all_labels[cb_index+1], layout=form_mainlabel_layout), all_checkboxes[cb_index+1]], layout=form_item_layout))

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='flex-start',
    width='70%'
))

In [6]:
#Function for creating config file
def writeconfigfile():
    f = open(configFileName, 'w')
    f.write('molecule name='+ molName + '\n')
    for cb_index in range(len(all_checkboxes)):
        if all_checkboxes[cb_index].value == True:
            f.write(all_properties[cb_index]+'=true\n')
        else:
            f.write(all_properties[cb_index]+'=false\n')
    f.close()

In [8]:
#If not using a configuration file that was created before, use the following checkboxes
#and then uncomment the cell containing the writeconfigfile function
#Parameters of the configuration file
#You may have to execute cell after selection once more
display(form)

In [9]:
#Use the following line if specific configuration file is to be created
writeconfigfile()

# Creating input file

In [10]:
createinput(molFileName,configFileName)

Creation of input file was successful


# Printing content of result file

In [11]:
%%bash -s "$yahFileName"
head $1
echo '...'
tail -n 15 $1

1bna
Molecular
Geometry
566
O 18.935 34.195 25.617
C 19.13 33.921 24.219
C 19.961 32.668 24.1
O 19.36 31.583 24.852
C 20.172 32.122 22.694
O 21.35 31.325 22.681
...
O 14.292 25.159 2.287
O 9.396 27.092 16.993
O 19.987 21.691 6.802
O 18.692 31.584 4.596
Charge
0

;Printing options
Print
distance matrix
orbital mapping
End_Print 

dump overlap
dump sparse


# Running extended Hückel calculations

In [12]:
huckel(yahFileName)

Extended Huckel calculations were successful
